In [1]:
## Generate a graph
using StatsBase
using Combinatorics

using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [2]:
# parameters

n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)

(::HypergraphModularity.var"#Ω#8"{HypergraphModularity.var"#Ω#3#9"{typeof(ω),Int64}}) (generic function with 1 method)

In [3]:
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

HypergraphModularity.hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [3, 2, 2, 3, 4, 3, 2, 3, 1, 5  …  2, 0, 2, 0, 1, 3, 2, 3, 1, 1]


In [6]:
## Run Louvains, Naive, and Faster version

# @time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

# mlou = modularity(H,Zlou,Ω;α=α0)
mlou2 = modularity(H,Zlou2,Ω;α=α0)

# mlou ≈ mlou2

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
  0.025965 seconds (237.99 k allocations: 11.397 MiB)


-852.96415856359334457758482740501182473025210128980688750743865966796875

In [7]:
Zsing = collect(1:n)

Ω̂_lou2 = estimateΩEmpirically(H, Zlou2;min_val=0)
Ω̂_sing = estimateΩEmpirically(H, Zsing; min_val=0)
Ω̂      = estimateΩEmpirically(H, Z; min_val=0)

Q_true = convert(Float64, modularity(H, Z, Ω̂;α=α0))
Q_sing = convert(Float64, modularity(H, Zsing, Ω̂_sing;α=α0))
Q_lou  = convert(Float64, modularity(H, Zlou2, Ω̂_lou2;α=α0))

println("The modularity of the true partition is $(round(Q_true,digits=3)).") # odd, not sure what's going on there
println("The modularity of the Louvain partition is $(round(Q_lou,digits=3)).")
println("The modularity of the singelton partition is $(round(Q_sing,digits=3)).")

The modularity of the true partition is -843.012.
The modularity of the Louvain partition is -120308.757.
The modularity of the singelton partition is -4.792187661e6.


In [8]:
# all with true parameters

println("The log-likelihood of the true partition is $(round(logLikelihood(H, Z, Ω, ϑ;α=α0),digits=3)).")
println("The log-likelihood of the Louvain partition is $(round(logLikelihood(H, Zlou2, Ω, ϑ;α=α0),digits=3)).")
println("The log-likelihood of the singleton partition is $(round(logLikelihood(H, Zsing, Ω, ϑ;α=α0),digits=3)).")

The log-likelihood of the true partition is -608.464.
The log-likelihood of the Louvain partition is -594.923.
The log-likelihood of the singleton partition is -608.671.


# Alternating Updates

In [9]:
# encouraging that this does indeed tend to decrease. I don't think it's required to be monotonically decreasing (need to check), so heuristically this looks ok-ish
# Ω̂ = buildΩ(estimateΩ(H, Z; min_val=10E-16); by_size=true)

Ω̂ = Ω

Z_ = copy(Z)

for i = 1:5
    Z_ = HyperLouvain(H,kmax,Ω̂;α=α0)
    Ω̂ = estimateΩEmpirically(H, Z_;min_val=0)
#     Ω̂  = buildΩ(estimateΩ(H, Z_; min_val=10E-16); by_size=true)
    println("The modularity is $(round(Float64(modularity(H, Z_, Ω̂;α=α0)), digits = 3))")
#     println("The log-likelihood of the Louvain partition is $(round(logLikelihood(H, Z_, Ω̂),digits=3)).")
end

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
The modularity is -120308.757
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
The modularity is -810.437
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
The modularity is -44826.149
Louvain Iteration 1
No nodes moved clusters
The modularity is -4.792187661e6
Louvain Iteration 1
No nodes moved clusters
The modularity is -4.792187661e6


In [10]:
Dict(p => Ω̂(p;α=α0) for p in partitions(3))

Dict{Array{Int64,1},Float64} with 3 entries:
  [3]       => 1.71429
  [1, 1, 1] => 0.0
  [2, 1]    => 0.6875